# Preparing the MIMIC 3 dataset

MIMIC3 can be downloaded from [here](https://mimic.physionet.org/gettingstarted/access/).

We use the same preprocessing as the authors of the GRU-ODE-Bayer paper and provide the final data preparation notebook that is missing in the original repository. First, execute the notebooks *admissions*, *outputs*, *labevents* and *prescriptions* provided [here](https://github.com/edebrouwer/gru_ode_bayes/tree/master/data_preproc/MIMIC).

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
import numpy as np

In [7]:
file_path="D:\Dataset\mimic-iv-3.0\mimic-iv-3.0"
outfile_path="D:\Dataset\mimic-iv-3.0\mimic-iv-3.0"

chart_df=pd.read_csv("D:\Dataset\mimic-iv-3.0\mimic-iv-3.0/chartevents_processed.csv")[["subject_id","hadm_id","charttime","valuenum","label"]]
admissions_df=pd.read_csv(f"D:\Dataset\mimic-iv-3.0\mimic-iv-3.0/admissions_processed.csv")[["hadm_id", "anchor_age"]]

lab_df=pd.read_csv("D:\Dataset\mimic-iv-3.0\mimic-iv-3.0/lab_processed.csv")[["subject_id","hadm_id","charttime","valuenum","label"]]
inputs_df=pd.read_csv("D:\Dataset\mimic-iv-3.0\mimic-iv-3.0/inputs_processed.csv")[["subject_id","hadm_id","charttime","amount","label"]]
outputs_df=pd.read_csv("D:\Dataset\mimic-iv-3.0\mimic-iv-3.0/outputs_processed.csv")[["subject_id","hadm_id","charttime","value","label"]]
presc_df=pd.read_csv("D:\Dataset\mimic-iv-3.0\mimic-iv-3.0/prescriptions_processed.csv")[["subject_id","hadm_id","charttime","dose_val_rx","drug"]]

C:\Users\huzep\AppData\Local\Temp\ipykernel_28276\1369867717.py:8: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  inputs_df=pd.read_csv("D:\Dataset\mimic-iv-3.0\mimic-iv-3.0/inputs_processed.csv")[["subject_id","hadm_id","charttime","amount","label"]]
C:\Users\huzep\AppData\Local\Temp\ipykernel_28276\1369867717.py:10: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  presc_df=pd.read_csv("D:\Dataset\mimic-iv-3.0\mimic-iv-3.0/prescriptions_processed.csv")[["subject_id","hadm_id","charttime","dose_val_rx","drug"]]


In [8]:
inputs_df["valuenum"]=inputs_df["amount"]
inputs_df.head()
inputs_df=inputs_df.drop(columns=["amount"]).copy()

outputs_df["valuenum"]=outputs_df["value"]
outputs_df=outputs_df.drop(columns=["value"]).copy()

presc_df["valuenum"]=presc_df["dose_val_rx"]
presc_df=presc_df.drop(columns=["dose_val_rx"]).copy()
presc_df["label"]=presc_df["drug"]
presc_df=presc_df.drop(columns=["drug"]).copy()



inputs_df["Origin"]="Inputs"
lab_df["Origin"]="Lab"
outputs_df["Origin"]="Outputs"
presc_df["Origin"]="Prescriptions"

# Use pd.concat instead of append (append is deprecated)
merged_df1 = pd.concat([inputs_df, lab_df], ignore_index=True).reset_index()
merged_df2 = pd.concat([merged_df1, chart_df], ignore_index=True)

# merged_df3 = pd.concat([merged_df2, outputs_df], ignore_index=True)
# merged_df3.drop(columns="index", inplace=True)
# merged_df = pd.concat([merged_df3, presc_df], ignore_index=True)

merged_df2.drop(columns="index", inplace=True)
merged_df = merged_df2

# assert(merged_df["label"].nunique()==(inputs_df["label"].nunique()+lab_df["label"].nunique()+outputs_df["label"].nunique()+
#                                       presc_df["label"].nunique() + chart_df["label"].nunique()))

In [9]:
merged_df = merged_df.merge(admissions_df)


In [10]:
del merged_df1
del merged_df2
# del merged_df3
del chart_df
del lab_df
del inputs_df
del outputs_df
import gc
gc.collect()
merged_df.head()

,subject_id,hadm_id,charttime,label,valuenum,Origin,anchor_age
0,10002114,27793700.0,2162-02-20 05:25:00,Magnesium Sulfate,1.0,Inputs,56
1,10002114,27793700.0,2162-02-20 05:55:00,Magnesium Sulfate,1.0,Inputs,56
2,10002114,27793700.0,2162-02-18 10:48:00,Potassium Chloride,10.0,Inputs,56
3,10002114,27793700.0,2162-02-18 11:18:00,Potassium Chloride,10.0,Inputs,56
4,10002114,27793700.0,2162-02-18 11:46:00,Potassium Chloride,10.0,Inputs,56


In [11]:
# df_copy = merged_df.copy()

# # Step 2: Modify the duplicated rows
# df_copy['label'] = 'anchor_age'  # Replace 'label' with 'anchor_age'
# df_copy['valuenum'] = df_copy['anchor_age']  # Replace 'valuenum' with 'anchor_age'

# # Step 3: Concatenate the original DataFrame with the modified DataFrame
# merged_df = pd.concat([merged_df, df_copy], ignore_index=True)


In [12]:
merged_df.head()

,subject_id,hadm_id,charttime,label,valuenum,Origin,anchor_age
0,10002114,27793700.0,2162-02-20 05:25:00,Magnesium Sulfate,1.0,Inputs,56
1,10002114,27793700.0,2162-02-20 05:55:00,Magnesium Sulfate,1.0,Inputs,56
2,10002114,27793700.0,2162-02-18 10:48:00,Potassium Chloride,10.0,Inputs,56
3,10002114,27793700.0,2162-02-18 11:18:00,Potassium Chloride,10.0,Inputs,56
4,10002114,27793700.0,2162-02-18 11:46:00,Potassium Chloride,10.0,Inputs,56


In [13]:
merged_df['charttime']=pd.to_datetime(merged_df["charttime"], format='%Y-%m-%d %H:%M:%S')
ref_time=merged_df.groupby("hadm_id")["charttime"].min()
merged_df.groupby("hadm_id").min().reset_index().to_csv(outfile_path + "/ref_time.csv", index=False)



merged_df_1=pd.merge(ref_time.to_frame(name="ref_time"),merged_df,left_index=True,right_on="hadm_id")
merged_df_1["time_stamp"]=merged_df_1["charttime"]-merged_df_1["ref_time"]
assert(len(merged_df_1.loc[merged_df_1["time_stamp"]<timedelta(hours=0)].index)==0)

C:\Users\huzep\AppData\Local\Temp\ipykernel_28276\195928190.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  merged_df.groupby("hadm_id").min().reset_index().to_csv(outfile_path + "/ref_time.csv", index=False)


In [14]:
del merged_df
merged_df_1["label"].nunique()

75

In [15]:
merged_df_1["label"].unique()

array(['Vancomycin', 'Hydromorphone (Dilaudid)', 'Hydralazine',
       'Heparin Sodium (Prophylaxis)', 'Cefazolin', 'Insulin - Regular',
       'Pantoprazole (Protonix)', 'Solution', 'Lactate', 'pH', 'pO2',
       'Bicarbonate', 'Chloride', 'Creatinine', 'Potassium', 'Sodium',
       'Urea Nitrogen', 'Hematocrit', 'Platelet Count',
       'White Blood Cells', 'Bilirubin', 'Heart Rate',
       'Non Invasive Blood Pressure systolic',
       'Non Invasive Blood Pressure diastolic', 'Respiratory Rate',
       'GCS - Eye Opening', 'GCS - Verbal Response',
       'GCS - Motor Response', 'Potassium Chloride', 'Gastric Meds',
       'Famotidine (Pepcid)', 'Furosemide (Lasix)',
       'Piperacillin/Tazobactam (Zosyn)', 'Propofol', 'Free Water',
       'Heparin Sodium', 'GT Flush', 'Dextrose 5%', 'Norepinephrine',
       'Albumin', 'Inspired O2 Fraction', 'Tidal Volume (observed)',
       'Plateau Pressure', 'Arterial CO2 Pressure', 'Acetaminophen-IV',
       'Magnesium Sulfate', 'Fentanyl', 'In

In [16]:
label_dict=dict(zip(list(merged_df_1["label"].unique()),range(len(list(merged_df_1["label"].unique())))))
merged_df_1["label_code"]=merged_df_1["label"].map(label_dict)

merged_df_short=merged_df_1[["hadm_id","valuenum","time_stamp","label_code","Origin", "anchor_age"]]

In [17]:
label_dict_df=pd.Series(merged_df_1["label"].unique()).reset_index()
label_dict_df.columns=["index","label"]
label_dict_df["label_code"]=label_dict_df["label"].map(label_dict)
label_dict_df.drop(columns=["index"],inplace=True)
label_dict_df.to_csv(outfile_path+"label_dict.csv")

In [18]:
# Convert valuenum to float, coercing errors to NaN, then drop rows with NaN if needed
merged_df_short["valuenum"] = pd.to_numeric(merged_df_short["valuenum"], errors='coerce')
merged_df_short = merged_df_short.dropna(subset=["valuenum"])

C:\Users\huzep\AppData\Local\Temp\ipykernel_28276\3486498781.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_short["valuenum"] = pd.to_numeric(merged_df_short["valuenum"], errors='coerce')


In [19]:
merged_df_short = merged_df_short.loc[merged_df_short["time_stamp"] < pd.Timedelta(minutes=2880)]
print("Number of patients considered :" + str(merged_df_short["hadm_id"].nunique()))

Number of patients considered :22018


In [20]:
merged_df_short.drop(["Origin"], axis=1, inplace=True)
merged_df_short

,hadm_id,valuenum,time_stamp,label_code,anchor_age
17987849,20000808.0,1.0,0 days 11:21:00,4,54
17987850,20000808.0,10.0,0 days 13:36:00,2,54
17987851,20000808.0,1.0,0 days 01:03:00,6,54
17987852,20000808.0,10.0,0 days 01:03:00,7,54
17987862,20000808.0,22.0,0 days 05:21:00,11,54
...,...,...,...,...,...
17930888,29999828.0,6.0,0 days 20:00:00,27,47
17930889,29999828.0,6.0,1 days 02:00:00,27,47
17930890,29999828.0,6.0,1 days 06:15:00,27,47
17930891,29999828.0,6.0,1 days 10:00:00,27,47


In [21]:
complete_df = None
complete_df = merged_df_short.copy()
labels = complete_df["label_code"].unique()
dynamic_labels_count = complete_df["label_code"].nunique()
value_columns = []
mask_columns  = []
for num in sorted(labels):
    name = "Value_label_" + str(num)
    name2 = "Mask_label_" + str(num)
    value_columns.append(name)
    mask_columns.append(name2)
    complete_df[name] = 0
    complete_df[name2] = 0
    complete_df[name] = complete_df[name].astype(float)

name = "Value_label_" + str(dynamic_labels_count)
name2 = "Mask_label_" + str(dynamic_labels_count)
value_columns.append(name)
mask_columns.append(name2)
complete_df[name] = 0
complete_df[name2] = 0
complete_df[name] = complete_df[name].astype(float)

C:\Users\huzep\AppData\Local\Temp\ipykernel_28276\2821995069.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_df[name2] = 0
C:\Users\huzep\AppData\Local\Temp\ipykernel_28276\2821995069.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_df[name] = 0
C:\Users\huzep\AppData\Local\Temp\ipykernel_28276\2821995069.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.

In [22]:
complete_df.dropna(inplace=True)
# 

In [18]:
for index, row in complete_df.iterrows():
    name = "Value_label_" + str(int(row["label_code"]))
    name2 = "Mask_label_" + str(int(row["label_code"]))
    complete_df.at[index, name] = row["valuenum"]
    complete_df.at[index, name2] = 1

    name = "Value_label_" + str(dynamic_labels_count)
    name2 = "Mask_label_" + str(dynamic_labels_count)
    complete_df.at[index, name] = row["anchor_age"]
    complete_df.at[index, name2] = 1


In [ ]:
# complete_df.drop(["valuenum", "label_code"], axis=1, inplace=True)
for c in ("valuenum", "label_code", "anchor_age"):
    if c in complete_df.columns:
        del complete_df[c]  
del merged_df_short
import gc
gc.collect()

complete_df = complete_df.groupby(["hadm_id", "time_stamp"], as_index=False).max()
for x in mask_columns:
    assert(len(complete_df.loc[complete_df[x]>1])==0)
complete_df

C:\Users\huzep\AppData\Local\Temp\ipykernel_23556\3446473389.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_df = complete_df.groupby(["hadm_id", "time_stamp"], as_index=False).max()
C:\Users\huzep\AppData\Local\Temp\ipykernel_23556\3446473389.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_df = complete_df.groupby(["hadm_id", "time_stamp"], as_index=False).max()


,hadm_id,time_stamp,anchor_age,Value_label_0,Mask_label_0,Value_label_1,Mask_label_1,Value_label_2,Mask_label_2,Value_label_3,...,Value_label_112,Mask_label_112,Value_label_113,Mask_label_113,Value_label_114,Mask_label_114,Value_label_115,Mask_label_115,Value_label_116,Mask_label_116
0,20000808.0,0 days 00:39:00,54,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,54.0,1
1,20000808.0,0 days 00:40:00,54,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,54.0,1
2,20000808.0,0 days 00:45:00,54,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,54.0,1
3,20000808.0,0 days 01:00:00,54,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,54.0,1
4,20000808.0,0 days 01:23:00,54,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,54.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725011,29999828.0,1 days 20:30:00,47,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,47.0,1
2725012,29999828.0,1 days 20:41:00,47,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,47.0,1
2725013,29999828.0,1 days 21:00:00,47,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,47.0,1
2725014,29999828.0,1 days 22:00:00,47,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,47.0,1


In [20]:
complete_df = complete_df.groupby(["hadm_id", "time_stamp"], as_index=False).max()
complete_df

C:\Users\huzep\AppData\Local\Temp\ipykernel_23556\3184240238.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_df = complete_df.groupby(["hadm_id", "time_stamp"], as_index=False).max()
C:\Users\huzep\AppData\Local\Temp\ipykernel_23556\3184240238.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_df = complete_df.groupby(["hadm_id", "time_stamp"], as_index=False).max()


,hadm_id,time_stamp,anchor_age,Value_label_0,Mask_label_0,Value_label_1,Mask_label_1,Value_label_2,Mask_label_2,Value_label_3,...,Value_label_112,Mask_label_112,Value_label_113,Mask_label_113,Value_label_114,Mask_label_114,Value_label_115,Mask_label_115,Value_label_116,Mask_label_116
0,20000808.0,0 days 00:39:00,54,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,54.0,1
1,20000808.0,0 days 00:40:00,54,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,54.0,1
2,20000808.0,0 days 00:45:00,54,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,54.0,1
3,20000808.0,0 days 01:00:00,54,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,54.0,1
4,20000808.0,0 days 01:23:00,54,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,54.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725011,29999828.0,1 days 20:30:00,47,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,47.0,1
2725012,29999828.0,1 days 20:41:00,47,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,47.0,1
2725013,29999828.0,1 days 21:00:00,47,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,47.0,1
2725014,29999828.0,1 days 22:00:00,47,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,47.0,1


In [21]:
unique_ids=np.arange(complete_df["hadm_id"].nunique())
np.random.shuffle(unique_ids)
d=dict(zip(complete_df["hadm_id"].unique(),unique_ids))

Unique_id_dict=pd.Series(complete_df["hadm_id"].unique()).reset_index().copy()
Unique_id_dict.columns=["index","hadm_id"]
Unique_id_dict["unique_id"]=Unique_id_dict["hadm_id"].map(d)
Unique_id_dict.to_csv("unique_id_dict.csv")

unique_id_df = pd.read_csv("unique_id_dict.csv")
d = dict(zip(unique_id_df["hadm_id"].values,unique_id_df["unique_id"].values))

In [22]:
admissions=pd.read_csv(f"D:\Dataset\mimic-iv-3.0\mimic-iv-3.0/admissions_processed.csv")
death_tags_s=admissions.groupby("hadm_id")["hospital_expire_flag"].unique().astype(int).to_frame().reset_index()
death_tags_df=death_tags_s.loc[death_tags_s["hadm_id"].isin(complete_df["hadm_id"])].copy()
death_tags_df["unique_id"]=death_tags_df["hadm_id"].map(d)
death_tags_df.sort_values(by="unique_id",inplace=True)
death_tags_df.rename(columns={"hospital_expire_flag":"Value"},inplace=True)
death_tags_df.set_index("unique_id", inplace=True)
# death_tags_df.drop("hadm_id", axis=1, inplace=True)
death_tags_df.to_csv("complete_death_tags.csv")

complete_df["unique_id"] = complete_df["hadm_id"].map(d)

In [23]:
complete_df.rename(columns={"unique_id": "ID", "time_stamp": "Time"}, inplace=True)
# complete_df.drop(["hadm_id"], axis=1, inplace=True)
# complete_df.set_index(["ID"], inplace=True)
complete_df

,hadm_id,Time,anchor_age,Value_label_0,Mask_label_0,Value_label_1,Mask_label_1,Value_label_2,Mask_label_2,Value_label_3,...,Mask_label_112,Value_label_113,Mask_label_113,Value_label_114,Mask_label_114,Value_label_115,Mask_label_115,Value_label_116,Mask_label_116,ID
0,20000808.0,0 days 00:39:00,54,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,54.0,1,10920
1,20000808.0,0 days 00:40:00,54,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,54.0,1,10920
2,20000808.0,0 days 00:45:00,54,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,54.0,1,10920
3,20000808.0,0 days 01:00:00,54,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,54.0,1,10920
4,20000808.0,0 days 01:23:00,54,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,54.0,1,10920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725011,29999828.0,1 days 20:30:00,47,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,47.0,1,4817
2725012,29999828.0,1 days 20:41:00,47,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,47.0,1,4817
2725013,29999828.0,1 days 21:00:00,47,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,47.0,1,4817
2725014,29999828.0,1 days 22:00:00,47,0.0,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,47.0,1,4817


In [24]:
complete_df.to_csv("full_dataset.csv")

In [1]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('full_dataset.csv')

# Remove the 'anchor_age' column
df.drop(columns=['anchor_age'], inplace=True)

# Save the modified DataFrame back to a CSV
df.to_csv('full_dataset.csv', index=False)
